In [ ]:
#library load
import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

In [ ]:
#시드 설정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)    #환경변수 설정 | 해시값 계산
    np.random.seed(seed)                        #동일한 시드 -> 항상 같은 난수
seed_everything(37)                             #Seed 고정

In [ ]:
#data load
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
#preprocessing
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID'])

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()  
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


In [ ]:
#결측치 제거
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
#변수 재설정
X = train_x
y = train_y
X_test0 = test_x

In [ ]:
# 데이터셋에서 0,1(적정),2 클래스 비율의 불균형 -> 비율에 맞게 k fold로 나눔 -> 오버피팅 방지를 통해 모델의 일반화 성능을 위함 
# 5개의 
skf = StratifiedKFold(n_splits=5, shuffle=True)
cv_accuracy = []
clf = CatBoostClassifier()

for train_index, valid_index in skf.split(train_x, train_y):
    X_train, X_valid = train_x.iloc[train_index], train_x.iloc[valid_index]
    y_train, y_valid = train_y.iloc[train_index], train_y.iloc[valid_index]
    

In [ ]:
# Class 불균형 문제 해결
# 오버샘플링 기법 : 소수 클래스(minority class) 샘플을 증가시켜 데이터셋의 클래스 균형을 맞추는 방법
from imblearn.over_sampling import BorderlineSMOTE

# X: feature 데이터, y: label 데이터
smote = BorderlineSMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(train_x, train_y)

In [ ]:
#필요한 라이브러리 호출
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# XGBoost 모델 생성
xgb = XGBClassifier(max_depth=3, 
                  learning_rate=0.1, 
                  n_estimators=100, 
                  random_state=42)

# Random Forest 모델 생성
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Voting 앙상블 모델 생성
estimators = [('xgb', xgb), ('rf', rf)]
ensemble = VotingClassifier(estimators=estimators, voting='soft')

# Grid Search를 통한 최적 하이퍼파라미터 탐색
params = {'xgb__max_depth': [3, 5, 7], 
          'xgb__learning_rate': [0.1, 0.01, 0.001], 
          'rf__n_estimators': [100, 200, 300]}

grid = GridSearchCV(estimator=ensemble, 
                    param_grid=params, cv=5, 
                    scoring='accuracy', 
                    n_jobs=-1)

grid.fit(X_resampled, y_resampled)

# 최적 하이퍼파라미터로 모델 학습 및 예측
ensemble = grid.best_estimator_
ensemble.fit(X_resampled, y_resampled)
y_pred = ensemble.predict(test_x)



In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission['Y_Class'] = y_pred

In [ ]:
submission.to_csv('./rf_day_feature_three.csv', index=False)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission['Y_Class'] = y_pred

In [ ]:
submission.to_csv('./ensemble_four.csv', index=False)

In [ ]:
"""# OOB Score 출력
print("OOB Score:", ensemble.oob_score_)"""